In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

In [12]:
table_data = pd.read_csv('data_table.csv', index_col='Сырье')
prices = pd.read_csv('prices.csv')

In [13]:
table_data.head()

,Запасы сырья,Затраты на стол,Затраты на стул,Затраты на бюро,Затраты на книжный шкаф
Сырье,,,,,
Доски 1,30,3,1,2,5
Доски 2,90,9,3,6,15
Труд. затраты,350,45,15,30,75


In [14]:
prices.head()

,Цена стола,Цена стула,Цена бюро,Цена книжного шкафа
0,210,70,140,350


$\Large F=c_{1}*x_{1}+c_{2}*x_{2}+c_{3}*x_{3}+c_{4}*x_{4} \xrightarrow{} max $

$\Large \begin{cases} a_{11}*x_1 + a_{12}*x_2 + a_{13}*x_3 + a_{14}*x_4 \leq b_1 \\ a_{21}*x_1 + a_{22}*x_2 + a_{23}*x_3 + a_{24}*x_4 \leq b_2 \\ a_{31}*x_1 + a_{32}*x_2 + a_{33}*x_3 + a_{34}*x_4 \leq b_3 \end{cases}$

In [15]:
def fill_c(prices: pd.DataFrame):
    """
    Считываем переменные целевой функции, умножив их на -1, и заполняем ими массив
    :param: prices: DataFrame with values
    :return: 1D array of values
    """
    negative_vals = prices.apply(lambda x: x*(-1))
    return negative_vals.values

In [16]:
def fill_a_ub(table_data: pd.DataFrame):
    """
    Считываем переменные левой части уравнения и записываем их в массив
    :param: table_data: DataFrame with values
    :return: 2D array of values
    """
    out_of_materials_stocks = table_data.drop('Запасы сырья', axis=1)
    return out_of_materials_stocks.values

In [17]:
def fill_b_ub(table_data: pd.DataFrame):
    """
    Считываем переменные правой части уравнения и записываем их в массив
    :param: table_data: DataFrame with values
    :return: 1D array of values
    """
    return table_data['Запасы сырья'].values

In [18]:
c = fill_c(prices)
a_ub = fill_a_ub(table_data)
b_ub = fill_b_ub(table_data)

$\Large F=210*x_{1}+70*x_{2}+140*x_{3}+350*x_{4} \xrightarrow{} max $

$\Large \begin{cases} 3*x_1 + 1*x_2 + 2*x_3 + 5*x_4 \leq 30 \\ 9*x_1 + 3*x_2 + 6*x_3 + 15*x_4 \leq 90 \\ 45*x_1 + 15*x_2 + 30*x_3 + 75*x_4 \leq 350 \end{cases}$

In [19]:
result = linprog(c=c, A_ub=a_ub, b_ub=b_ub)

In [20]:
result

     con: array([], dtype=float64)
     fun: -1633.3333311874344
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([6.66666670e+00, 2.00000001e+01, 4.59835519e-07])
  status: 0
 success: True
       x: array([1.82978386, 3.57887523, 2.47049167, 1.86482463])